In [1]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_jpace7" #please modify so you store data in your collection
my_char = 'i'

# beginning page index
begin = "1"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]


beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

begin_source_url = "https://sourceforge.net/directory/?q=" + my_char + "&sort=name&page="
rest_source_url = "https://sourceforge.net/rest/p/"

gleft = 20 #try not to get banned this time

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

def project_status(url):
    r = requests.get(url)
    if r.status_code == 200:
        return True
    return False

def get_source(url, coll, rest):
    page = 1
    count = 0
    while True:
        resp = requests.get(url + str(page))
        text = resp.text
        soup = BeautifulSoup(text, 'html.parser')
        
        if re.search('No results found.', soup.get_text()):
            return

        for link in soup.find_all(class_ = "project-icon", href = True):
            name = re.findall('/projects/([A-Za-z0-9\-]*)', link.get('href'))
            #if name: name = name[0]
            #else: name = None
            name = name[0] if name else None
            if name is not None and name.lower().startswith(my_char):
                resp = requests.get(rest + name)
                if project_status(rest + name):
                    info = json.loads(resp.text)
                    info['forge'] = 'sourceforge'
                    coll.insert_one(info)
                    count += 1
                    if count >= 50:
                        return
        page += 1
    return

# send queries and extract urls 
def get_git(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    count = 0

    try:
        #print(url)
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            #put the project into our list. If we have 50, stop
            for el in array:
                #print(el['name'])
                if el['name'].lower().startswith(my_char):
                    if project_status(el['http_url_to_repo']):
                        print('Step 2')
                        count += 1
                        el['forge'] = 'gitlab'
                        coll.insert_one(el)
                        if count >= 50:
                            return
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            #print(el['name'])
                            if el['name'].lower().startswith(my_char):
                                if project_status(el['http_url_to_repo']):
                                    count += 1
                                    el['forge'] = 'gitlab'
                                    coll.insert_one(el)
                                    if count >= 50:
                                        return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
#start retrieving
print('Started')
get_git(beginurl,coll)
print('Halfway')
get_source(begin_source_url, coll, rest_source_url)
print('Finished')

Started
redro
iocfx
Step 2
encyclopedia
obelisk-ob1
bin
workspace
virglrenderer
hellhound
MyORM
MBTA
libnice
My dotfiless
React SSR
www-gitlab-com
pkg-config
electricite-chisloup
fontconfig
gam345highscores
ProjetManoir
data-analysys
WorldCup
Chat exercise
stm32f103_tft_8bit
junit5-jupiter-starter-m2e-java11-eclipse
Teste
omim
challenge
libsignon-glib
Algebra II
entropy-automation
tasks_mem_analysis
protobuf-java-parser
sparkbot-admin-web
EQ
Weekly Mood
C Utils
panalpina-web
kitto-core
socialsupacrew.github.io
NowPlayingList
NFC-Clock-Project
dresscode
doom.d
gsignond
Programming Challenges
GravadorTelaFFMPEG
golang-demo
inf3135-183-exemples
Step 2
prueba-app
Luca
GraPong
WirtualnySwiatCpp
Pack4Travel
ChurchMap
cygnus
WishingMachine
pylarization
ZPO lab03
site
E-commerce platform
salt_sandbox
CS_Retour_Project
Mebe 2
sql-challenge
map reduce word counter
Challenge Front End
gestion-programme-frontend
First Assignment - DevOps
Challenge Back End
router-fn
router
ExPra
html-fn
core-fn
ht

reservantes
4DG
ulmo
MO Conception de site web
MiProyecto
parent-project
FIRST_COMMIT
El Dorsal
thuctap
bash-script
testForMtech
Magic Child
tac2-2018
cpp-project-template
portfolio
testjson
Faircoin donation widget
statscloud.io-rails-example
TheEmptyProject
FinalCalci
statscloud.io-ruby-example
Larave Ecommerce
frontend_pec1
abrahaminetianbor
RestAPI
cybersecuritynlp
demo1
bikes
tripleo-standalone-edge
webscraping
smart-xml-analyzer-ae
myNewCalendar
Jimmys_akazoo
kotlin
igcprox
igcparse
Mats After Effects Panel
Assignment1 DevOps
come-back
my-little-rocket
timelapse-controller
Syntel_Demo
BottomSheetButton
GitLab Community Edition
mock-test-react
ESO
util-auth
Swiper-iframe
inappbrowser
gulp-phonegap-build
pyquant-demo-201810
learn-go-in-3-hours
FE Sesi 7
primasaver
codeborn-test
crm-service-registry
u-test
Mats Unity Template
test2
Mats Godot Template
File system dockerfile
Diusframi
Nespress
talks
Misty Hats   3Code Jam  Tests
paw1
thpx_blog
zavier-hexo
ok
virtio-fs.gitlab.io
test2

IVY-Tech-New
IvyTech
iupr
ITSH
ITSG
ITS
VUE_KTTEAM-01
ITP
ITM_M
VUE_KTTEAM-01
ITMV
ITM
ITMM
IsleOfWhite
production
production_outsourcing
CNG
ISHRAT
ISB
production_routing
IPS
iphoneXissue
sdfsdfgsdgsdg
IPCPRC
Iowa-Lakes-Bluelight
INMANTEC
InvestmentMatter
IMSNOIDA
IMSNOIDA
Flappy Bird - Phaser
production_split
PS3 MOD MENU - illicit remake
production_work
production_work_timesheet
blogRails
product_measurements
Liam
unendlichkategorien
product_price_list
canvas-lms-docker
Imadedev
Imperial
ImadeCCBC
product_price_list_dates
Imade-NSC
Imade-global
Imade-Belmont
Imade-Framework
ILSDEMO
Illinois-College-Emergency
nodemonorepo.mirror2
product_price_list_parent
IITM
IITKGP
IITJ
project
IITDelhi
project_invoice
consemulJS
project_plan
project_revenue
Hugo-Basic-theme
purchase
purchase_invoice_line_standalone
651_citi_test_bd
purchase_request
purchase_request_quotation
IITBHU
Halfway
Finished


In [2]:
for doc in coll.find({}):
    print(doc)

{'default_branch': 'develop', 'avatar_url': None, 'path': 'iocfx', 'id': 8878938, 'created_at': '2018-10-15T20:13:21.679Z', 'web_url': 'https://gitlab.com/bbmsoft.net/iocfx', 'readme_url': 'https://gitlab.com/bbmsoft.net/iocfx/blob/develop/README.md', 'forks_count': 0, 'forge': 'gitlab', 'name_with_namespace': 'bbmsoft.net / iocfx', 'http_url_to_repo': 'https://gitlab.com/bbmsoft.net/iocfx.git', 'namespace': {'path': 'bbmsoft.net', 'id': 3751362, 'kind': 'group', 'full_path': 'bbmsoft.net', 'parent_id': None, 'name': 'bbmsoft.net'}, 'path_with_namespace': 'bbmsoft.net/iocfx', 'description': 'A minimal Inversion of Control framework designed to improve the usability of JavaFX when running it from within OSGi.', 'name': 'iocfx', 'tag_list': [], 'star_count': 0, 'last_activity_at': '2018-10-15T20:13:21.679Z', 'ssh_url_to_repo': 'git@gitlab.com:bbmsoft.net/iocfx.git', '_id': ObjectId('5bc4f517c6a5a63936288de6')}
{'default_branch': 'master', 'avatar_url': None, 'path': 'inf3135-183-exemples